# feature analysis result - already exist in the previous result file

We use WL or CCA to do the
1. inter-subject gesture classification
2. Few-shot adaptation

WL로 가자

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os, sys, random
sys.path.append('../../Shared')
import processing, Model, Visualization, modules, Adaptation
from sklearn.model_selection import train_test_split



def import_data(subject, current_rep, db_name, feat_idx):
    path = f'C:/Users/hml76/Desktop/UMN/Research/5th Federated Prototype Learning/Dataset/{db_name}/'

    x_train = np.array(pd.read_csv(path + 'Sub{}/Sub{}_Rep{}_data.csv'.format(subject, subject, current_rep)))
    y_train = np.array(pd.read_csv(path + 'Sub{}/Sub{}_Rep{}_label.csv'.format(subject, subject, current_rep)))
    indices = np.random.permutation(len(x_train)) #shuffle
    x_train, y_train = x_train[indices], y_train[indices]

    x_train = x_train.reshape(len(x_train), 10, 11)
    x_train = x_train[:, :, feat_idx]
    x_train = x_train.reshape(len(x_train), 10, 1)

    return x_train, y_train

def ready_data_per_subject_all_reps(subject, Data, downsample, downsample_size, feat_idx):
    X, y = [], []
    for rep in range(1,11):
        X_tmp, y_tmp = import_data(subject, rep, Data, feat_idx)

        X.append(X_tmp)
        y.append(y_tmp)
    X = np.concatenate(X, axis=0)
    y = np.concatenate(y, axis=0)
    y = np.array([np.argmax(y[i]) for i in range(len(y))])

    if downsample:
        idx = np.random.choice(X.shape[0], size=downsample_size, replace=False)
        X, y = X[idx], y[idx]

    return X, y

In [2]:
#sub_lst = random.sample(range(1, 28), 20)
sub_lst = [23, 20, 27, 24, 14, 6, 17, 3, 9, 2, 1, 12, 5, 22, 16, 18, 13, 11, 21, 26]

# Baseline - LOSO

Random 10 subjects

In [ ]:
Data_name = "DB1"
F = 0

Sub_all = []

for train_subject in sub_lst:
    print(f"Testing subject {train_subject}")
    X_train, y_train = ready_data_per_subject_all_reps(subject=train_subject, Data=Data_name, downsample=True, downsample_size=25000, feat_idx=F)

    model = Model.build_model_1D(input_shape=X_train.shape[1:], num_classes=len(np.unique(y_train)))
    history = model.fit(X_train, y_train, epochs=100, batch_size=256, verbose=0)
    plt.plot(history.history['accuracy'])
    plt.show()

    Sub_acc = []
    for test_subject in sub_lst:
        if test_subject != train_subject:
            X_test, y_test = ready_data_per_subject_all_reps(subject=test_subject, Data=Data_name, downsample=True, downsample_size=10000, feat_idx=F)
            loss, acc = model.evaluate(X_test, y_test, verbose=0)
            print(f"\t Testing subject {test_subject} : acc {acc*100:.2f}%")
            Sub_acc.append(acc)
        else:
            continue
    Sub_all.append(Sub_acc)

In [14]:
Sub_all

[[0.031199999153614044,
  0.11060000211000443,
  0.08250000327825546,
  0.0421999990940094,
  0.06120000034570694,
  0.06210000067949295,
  0.08030000329017639,
  0.024900000542402267,
  0.03269999846816063,
  0.02930000051856041,
  0.06270000338554382,
  0.04390000179409981,
  0.05999999865889549,
  0.03790000081062317,
  0.055799998342990875,
  0.037300001829862595,
  0.027899999171495438,
  0.04650000110268593,
  0.08940000087022781],
 [0.017799999564886093,
  0.023600000888109207,
  0.031599998474121094,
  0.04639999940991402,
  0.045099999755620956,
  0.05180000141263008,
  0.0674000009894371,
  0.02539999969303608,
  0.027300000190734863,
  0.06629999727010727,
  0.051600001752376556,
  0.03020000085234642,
  0.07029999792575836,
  0.050999999046325684,
  0.053199999034404755,
  0.07000000029802322,
  0.05860000103712082,
  0.023499999195337296,
  0.0421999990940094],
 [0.06340000033378601,
  0.03779999911785126,
  0.05790000036358833,
  0.03759999945759773,
  0.02019999921321869

In [15]:
baseline = pd.DataFrame(Sub_all)
baseline.shape

(20, 19)

In [16]:
baseline.to_csv('./DB1_baseline.csv', index=False)

# Inter-subject incremental

In [3]:
Data_name = "DB1"
Sub_all = []

for test_subject in sub_lst:
    print(f"Testing subject {test_subject}")
    X_test, y_test = ready_data_per_subject_all_reps(subject=test_subject, Data=Data_name, downsample=True, downsample_size=10000, feat_idx=1)

    Sub_acc, X_train, y_train = [], [], []
    for train_subject in sub_lst:
        if train_subject != test_subject:
            X_train_tmp, y_train_tmp = ready_data_per_subject_all_reps(subject=train_subject, Data=Data_name, downsample=True, downsample_size=25000, feat_idx=1)
            X_train.append(X_train_tmp)
            y_train.append(y_train_tmp)

            X_train_arr, y_train_arr = np.concatenate(X_train, axis=0), np.concatenate(y_train, axis=0)
            print("\t", X_train_arr.shape, y_train_arr.shape)

            model = Model.build_model_1D(input_shape=X_train_arr.shape[1:], num_classes=len(np.unique(y_train_arr)))
            model.fit(X_train_arr, y_train_arr, epochs=100, batch_size=256, verbose=0)
            loss, acc = model.evaluate(X_test, y_test, verbose=0)
            print(f"\t Added subject {train_subject} : acc {acc*100:.2f}%")
            Sub_acc.append(acc)

    Sub_all.append(Sub_acc)

Testing subject 23
	 (25000, 10, 1) (25000,)
	 Added subject 20 : acc 3.30%
	 (50000, 10, 1) (50000,)
	 Added subject 27 : acc 5.41%
	 (75000, 10, 1) (75000,)
	 Added subject 24 : acc 6.58%
	 (100000, 10, 1) (100000,)
	 Added subject 14 : acc 6.70%
	 (125000, 10, 1) (125000,)
	 Added subject 6 : acc 7.44%
	 (150000, 10, 1) (150000,)
	 Added subject 17 : acc 8.07%
	 (175000, 10, 1) (175000,)
	 Added subject 3 : acc 9.60%
	 (200000, 10, 1) (200000,)
	 Added subject 9 : acc 8.57%
	 (225000, 10, 1) (225000,)
	 Added subject 2 : acc 9.23%
	 (250000, 10, 1) (250000,)
	 Added subject 1 : acc 8.55%
	 (275000, 10, 1) (275000,)
	 Added subject 12 : acc 8.95%
	 (300000, 10, 1) (300000,)
	 Added subject 5 : acc 8.94%
	 (325000, 10, 1) (325000,)
	 Added subject 22 : acc 8.93%
	 (350000, 10, 1) (350000,)
	 Added subject 16 : acc 7.23%
	 (375000, 10, 1) (375000,)
	 Added subject 18 : acc 9.23%
	 (400000, 10, 1) (400000,)
	 Added subject 13 : acc 9.33%
	 (425000, 10, 1) (425000,)
	 Added subject 11 : 

In [4]:
acccc = pd.DataFrame(Sub_all)
acccc.to_csv('./DB1_inter_sub_accumulate.csv', index=False)
acccc.shape

(20, 19)

# Few-shot adaptation